In [350]:
from pprint import pprint
import copy
from dictdiffer import diff

group = {}
transactions = {}

def normalizeGroup(dictionary: dict):
    for p1n, p1d in dictionary.items():
        p1d = p1d["balances"]
        for p2n, p2d in dictionary.items():
            p2d = p2d["balances"]
            if p1n != p2n:
                if p1n not in p2d["owes"]:
                    p2d["owes"][p1n] = 0
                if p2n not in p1d["owes"]:
                    p1d["owes"][p2n] = 0
                if p1n not in p2d["owed_by"]:
                    p2d["owed_by"][p1n] = 0
                if p2n not in p1d["owed_by"]:
                    p1d["owed_by"][p2n] = 0

    return dictionary


def addPerson(name):
    group[name] = {}
    group[name]["balances"] = {
        "owes": {},
        "owed_by": {}
    }
    # add total paid by and total owed to
    group[name]["total_owed"] = 0
    group[name]["total_owing"] = 0
    group[name]["balance"] = 0
    normalizeGroup(group)

def createGroup():
    group.clear()
    transactions.clear()
    addPerson("person1")
    addPerson("person2")
    addPerson("person3")

def printGroup():
    for person, data in group.items():
        print(f"{person.upper()}: [Balance: {data['balance']}]")
        print(f"\tOwes to others: [{data['total_owed']}]")
        for owes, amount in data["balances"]["owes"].items():
            print(f"\t\t-> {owes}: {amount}")
        print(f"\tOwed by others: [{data['total_owed']}]")
        for owed_by, amount in data["balances"]["owed_by"].items():
            print(f"\t\t-> {owed_by}: {amount}")

createGroup()
pprint(group)

{'person1': {'balance': 0,
             'balances': {'owed_by': {'person2': 0, 'person3': 0},
                          'owes': {'person2': 0, 'person3': 0}},
             'total_owed': 0,
             'total_owing': 0},
 'person2': {'balance': 0,
             'balances': {'owed_by': {'person1': 0, 'person3': 0},
                          'owes': {'person1': 0, 'person3': 0}},
             'total_owed': 0,
             'total_owing': 0},
 'person3': {'balance': 0,
             'balances': {'owed_by': {'person1': 0, 'person2': 0},
                          'owes': {'person1': 0, 'person2': 0}},
             'total_owed': 0,
             'total_owing': 0}}


In [351]:
printGroup()

PERSON1: [Balance: 0]
	Owes to others: [0]
		-> person2: 0
		-> person3: 0
	Owed by others: [0]
		-> person2: 0
		-> person3: 0
PERSON2: [Balance: 0]
	Owes to others: [0]
		-> person1: 0
		-> person3: 0
	Owed by others: [0]
		-> person1: 0
		-> person3: 0
PERSON3: [Balance: 0]
	Owes to others: [0]
		-> person1: 0
		-> person2: 0
	Owed by others: [0]
		-> person1: 0
		-> person2: 0


In [352]:
def add_transaction(who_got_how_much, who_paid_how_much):
    # the transaction total is the sum of all the values in the who_got_how_much
    transaction_total = sum(who_got_how_much.values())
    pay_total = sum(who_paid_how_much.values())

    # the transaction total must equal the sum of the values in the who_paid_how_much
    if pay_total != transaction_total:
        raise Exception(f"ERROR: pay total {pay_total} does not equal transaction total {transaction_total}")

    transaction = {"who_got_how_much": who_got_how_much,
                   "who_paid_how_much": who_paid_how_much,
                   "transaction_total": transaction_total,
                   "pay_total": pay_total,
                   "prior_state": {}}

    # copy the group's previous state from the previous transaction
    if len(transactions) > 0:
        transaction["prior_state"] = transactions[-1]["prior_state"].copy()
    else:
        for person in group:
            transaction["prior_state"][person] = copy.deepcopy(group[person])

    g1 = copy.deepcopy(transaction["prior_state"])
    # print("=" * 80)
    # print("# Prior State:")
    # pprint(g1)
    prior_state = transaction["prior_state"]
    for p1, owing in who_got_how_much.items():
        for p2, amt_paid in who_paid_how_much.items():

            if p1 != p2:
                value = amt_paid / transaction_total * owing

                transaction["prior_state"][p1]["balances"]["owes"][p2] += value
                transaction["prior_state"][p2]["balances"]["owed_by"][p1] += value

                owes = transaction["prior_state"][p1]["balances"]["owes"][p2]
                owed_by = transaction["prior_state"][p2]["balances"]["owed_by"][p1]
                owes_minus_owed_by = owes - owed_by

    # Calculate the total owing, total owed, and balance for each person
    for p, v in transaction["prior_state"].items():
        # for each person, calculate the new owed and owed by taking the difference and set it to the owes and owed_by depending on the sign
        for p1 in v["balances"]["owes"]:
            owes = v["balances"]["owes"][p1]
            owed_by = v["balances"]["owed_by"][p1]
            owes_minus_owed_by = owes - owed_by
            if owes_minus_owed_by > 0:
                v["balances"]["owes"][p1] = owes_minus_owed_by
                v["balances"]["owed_by"][p1] = 0
            elif owes_minus_owed_by < 0:
                v["balances"]["owes"][p1] = 0
                v["balances"]["owed_by"][p1] = -owes_minus_owed_by
            else:
                v["balances"]["owes"][p1] = 0
                v["balances"]["owed_by"][p1] = 0

        transaction["prior_state"][p]["total_owing"] = sum(v["balances"]["owes"].values())
        transaction["prior_state"][p]["total_owed"] = sum(v["balances"]["owed_by"].values())
        transaction["prior_state"][p]["balance"] = transaction["prior_state"][p]["total_owed"] - transaction["prior_state"][p]["total_owing"]

    def get_diff():
        g2 = copy.deepcopy(transaction["prior_state"])
        # pprint(g2)
        # Only print the fields in the new state that have changed from the group's prior state
        difference = ""
        for person in group:
            diff_dict = list(diff(g1[person], g2[person]))
            if len(diff_dict) > 0:
                difference += f"{person.upper()}:\n"
                for d in diff_dict:
                    d3 = d[2][1] - d[2][0]
                    info = "[=]"
                    if d3 > 0:
                        info = "+"
                    elif d3 < 0:
                        info = "-"
                    difference += f"\t[{info}] {d}\n"
            else:
                difference += f"{person}: No change\n"
        return difference

    # Update the group's balances with the new state
    group.update(transaction["prior_state"])
    return get_diff()

def assertDiff(diff_expected, diff_actual, printFull=False):
    if printFull:
        print("=" * 80)
        print("# True Diff:")
        print(diff_expected)
        print("=" * 80)
        print("# Actual Diff:")
        print(diff_actual)
        print("=" * 80)

    def cmp_helper(exp, act):
        de = exp.split("\n")
        da = act.split("\n")
        for i in range(len(de)):
            if de[i] != da[i]:
                print(f"ERROR: (Expected) `{de[i]}` != `{da[i]}` (Actual)")
                return False
        return True

    assert cmp_helper(diff_expected, diff_actual)
    print("PASSED!!!")

In [353]:
t = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person1': 20,
})
t_correct = """person1: No change
person2: No change
person3: No change
"""
assertDiff(t_correct, t)

PASSED!!!


In [354]:
t = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 20,
})
t_correct = """PERSON1:
	[+] ('change', 'balances.owes.person2', (0, 20.0))
	[+] ('change', 'total_owing', (0, 20.0))
	[-] ('change', 'balance', (0, -20.0))
PERSON2:
	[+] ('change', 'balances.owed_by.person1', (0, 20.0))
	[+] ('change', 'total_owed', (0, 20.0))
	[+] ('change', 'balance', (0, 20.0))
person3: No change
"""
assertDiff(t_correct, t)

PASSED!!!


In [355]:
t = add_transaction(who_got_how_much={
    'person2': 20,
}, who_paid_how_much={
    'person1': 20,
})

In [356]:
t = add_transaction(who_got_how_much={
    'person1': 20,
}, who_paid_how_much={
    'person2': 10,
    'person3': 10,
})

In [357]:
t = add_transaction(who_got_how_much={
    'person2': 10,
    'person3': 10,
}, who_paid_how_much={
    'person1': 20,
})

In [358]:
t = add_transaction(who_got_how_much={
    'person2': 30,
}, who_paid_how_much={
    'person2': 30,
})

In [359]:
t = add_transaction(who_got_how_much={
    'person2': 30,
    'person3': 30,
}, who_paid_how_much={
    'person2': 30,
    'person3': 30,
})